In [1]:
import time
from time import sleep

from random import randint

import pandas as pd

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import *

from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# transformando o arquivo excel em dataframe
cartas_df = pd.read_excel('excel/lista_cartas_magic_com_edicao.xlsx')

# eliminando as duplicatas (cartas com o mesmo nome e edições diferentes)
cartas_df = cartas_df.drop_duplicates(
    subset='nome_portugues', keep="first").reset_index(drop=True)

In [3]:
cartas_df

,nome_portugues,nome_ingles,edicao
0,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica
1,Proteção Oscilante,Flickering Ward,Tempestade
2,Desenterrar,Unearth,O Legado de Urza
3,Aluren,Aluren,Tempestade
4,Tutor Místico,Mystical Tutor,Sexta Edição Classica
5,Abertura Temporal,Temporal Aperture,A Saga de Urza
6,Aranha Escondida,Hidden Spider,A Saga de Urza
7,Guerra Justa,Righteous War,Visões
8,Bênção de Cho-Manno,Cho-Manno's Blessing,Máscara de Mercádia
9,Juramento dos Regentes,Oath of Lieges,Êxodo


In [4]:
# algumas configurações para o webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--proxy-server='direct://'")
chrome_options.add_argument("--proxy-bypass-list=*")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--ignore-certificate-errors')

# instanciando a versão do webdriver que será instalada na máquina local
service = Service(ChromeDriverManager().install())

# instanciando o webdriver
driver = webdriver.Chrome(service=service, options=chrome_options)

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 76.7MB/s]


# Extração

In [5]:
# efetuando busca no site
cartas_web_dic = []

for portugues, ingles in zip(cartas_df['nome_portugues'], cartas_df['nome_ingles']):

    driver.get(f"https://www.ligamagic.com/?view=cards/card&card={portugues}&aux={ingles}")

    sleep(randint(2, 4))

    action = ActionChains(driver)

    # selecionar a edição
    edcard = 0

    while True:
        try:
            edicao_menu = driver.find_element(
                By.CSS_SELECTOR, f"#edcard_{edcard}> img:nth-child(1)")
            action.move_to_element(edicao_menu).click().perform()

        except NoSuchElementException:
            break
        else:
            edcard += 1

        # pegando algumas características das cartas
        nome_portugues = portugues
        nome_ingles = ingles
        edicao = driver.find_element(By.XPATH, '//*[@id="ed-nome"]/a').text
        artista = driver.find_element(By.XPATH, '//*[@id="ed-artista"]/a').text
        raridade = driver.find_element(
            By.XPATH, '//*[@id="ed-raridade"]/a').text
        valor_medio_srt = driver.find_element(
            By.XPATH, '//*[@id="card-info"]/div[5]/div[2]/div/div[4]').text

        # exibindo nome (português e inglês), edição, artista, raridade e o valor médio
        print(f"{edcard}. {nome_portugues} | {nome_ingles} | {edicao} | {artista} | {raridade} | {valor_medio_srt}")
        
        # criando o dicionário
        cartas_web_dic.append({
            'nome_portugues': nome_portugues,
            'nome_ingles': nome_ingles,
            'edicao': edicao,
            'artista': artista,
            'raridade': raridade,
            'valor_medio_srt': valor_medio_srt
        })

1. Tutor Vampírico | Vampiric Tutor  | Dominaria Remastered (Retro Frame) | Raymond Swanland | Mítica | R$ 262,07
2. Tutor Vampírico | Vampiric Tutor  | Dominaria Remastered (Borderless) | Richard Kane Ferguson | Mítica | R$ 294,34
3. Tutor Vampírico | Vampiric Tutor  | Dominaria Remastered | Raymond Swanland | Mítica | R$ 234,68
4. Tutor Vampírico | Vampiric Tutor  | Commander Legends (Variantes) | Raymond Swanland | Mítica | R$ 313,66
5. Tutor Vampírico | Vampiric Tutor  | Commander Legends | Raymond Swanland | Mítica | R$ 251,96
6. Tutor Vampírico | Vampiric Tutor  | The List | Raymond Swanland | Mítica | R$ 258,95
7. Tutor Vampírico | Vampiric Tutor  | Eternal Masters | Raymond Swanland | Mítica | R$ 266,42
8. Tutor Vampírico | Vampiric Tutor  | Sexta Edição Classica | Gary Leach | Rara | R$ 280,33
9. Tutor Vampírico | Vampiric Tutor  | Visões | Gary Leach | Rara | R$ 289,11
10. Tutor Vampírico | Vampiric Tutor  | Vintage Masters | Gary Leach | Rara | R$ 149,99
11. Tutor Vampírico 

In [6]:
# convertendo o dicionário para dataframe
cartas_web_df = pd.DataFrame().from_dict(cartas_web_dic)

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt
0,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered (Retro Frame),Raymond Swanland,Mítica,"R$ 262,07"
1,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered (Borderless),Richard Kane Ferguson,Mítica,"R$ 294,34"
2,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered,Raymond Swanland,Mítica,"R$ 234,68"
3,Tutor Vampírico,Vampiric Tutor,Commander Legends (Variantes),Raymond Swanland,Mítica,"R$ 313,66"
4,Tutor Vampírico,Vampiric Tutor,Commander Legends,Raymond Swanland,Mítica,"R$ 251,96"
5,Tutor Vampírico,Vampiric Tutor,The List,Raymond Swanland,Mítica,"R$ 258,95"
6,Tutor Vampírico,Vampiric Tutor,Eternal Masters,Raymond Swanland,Mítica,"R$ 266,42"
7,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 280,33"
8,Tutor Vampírico,Vampiric Tutor,Visões,Gary Leach,Rara,"R$ 289,11"
9,Tutor Vampírico,Vampiric Tutor,Vintage Masters,Gary Leach,Rara,"R$ 149,99"


In [7]:
# fechando o site
time.sleep(5)
driver.close()

# Transformação

In [8]:
# transformando o valor médio string para float
cartas_web_df['valor_medio'] = list(map(lambda x : x.split(" ")[1].replace(".", "").replace(",", "."), cartas_web_df['valor_medio_srt']))
cartas_web_df['valor_medio'] = list(map(lambda x : float(x), cartas_web_df['valor_medio']))

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio
0,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered (Retro Frame),Raymond Swanland,Mítica,"R$ 262,07",262.07
1,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered (Borderless),Richard Kane Ferguson,Mítica,"R$ 294,34",294.34
2,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered,Raymond Swanland,Mítica,"R$ 234,68",234.68
3,Tutor Vampírico,Vampiric Tutor,Commander Legends (Variantes),Raymond Swanland,Mítica,"R$ 313,66",313.66
4,Tutor Vampírico,Vampiric Tutor,Commander Legends,Raymond Swanland,Mítica,"R$ 251,96",251.96
5,Tutor Vampírico,Vampiric Tutor,The List,Raymond Swanland,Mítica,"R$ 258,95",258.95
6,Tutor Vampírico,Vampiric Tutor,Eternal Masters,Raymond Swanland,Mítica,"R$ 266,42",266.42
7,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 280,33",280.33
8,Tutor Vampírico,Vampiric Tutor,Visões,Gary Leach,Rara,"R$ 289,11",289.11
9,Tutor Vampírico,Vampiric Tutor,Vintage Masters,Gary Leach,Rara,"R$ 149,99",149.99


In [9]:
# calculando a comissão do mercado livre

def comissao_ml(valor_medio) -> int:
    ''' 
    calculo de porcentagem (ml) e valor final do produto:
    - valores maiores ou iguais a R$ 79,00 a comissão do ML é de 11,5%
    - valores menores que R$ 79,00 a comissão do ML é de 11,5% + R$ 5,00
    '''
    if valor_medio >= 79:
        porcentagem_ml = (0.115 * valor_medio)
    else:
        porcentagem_ml = (0.115 * valor_medio) + 5

    valor_ml = round((porcentagem_ml + valor_medio), 2)
    
    return valor_ml


cartas_web_df['valor_ml'] = list(map(lambda x: comissao_ml(x), cartas_web_df['valor_medio']))

cartas_web_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered (Retro Frame),Raymond Swanland,Mítica,"R$ 262,07",262.07,292.21
1,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered (Borderless),Richard Kane Ferguson,Mítica,"R$ 294,34",294.34,328.19
2,Tutor Vampírico,Vampiric Tutor,Dominaria Remastered,Raymond Swanland,Mítica,"R$ 234,68",234.68,261.67
3,Tutor Vampírico,Vampiric Tutor,Commander Legends (Variantes),Raymond Swanland,Mítica,"R$ 313,66",313.66,349.73
4,Tutor Vampírico,Vampiric Tutor,Commander Legends,Raymond Swanland,Mítica,"R$ 251,96",251.96,280.94
5,Tutor Vampírico,Vampiric Tutor,The List,Raymond Swanland,Mítica,"R$ 258,95",258.95,288.73
6,Tutor Vampírico,Vampiric Tutor,Eternal Masters,Raymond Swanland,Mítica,"R$ 266,42",266.42,297.06
7,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 280,33",280.33,312.57
8,Tutor Vampírico,Vampiric Tutor,Visões,Gary Leach,Rara,"R$ 289,11",289.11,322.36
9,Tutor Vampírico,Vampiric Tutor,Vintage Masters,Gary Leach,Rara,"R$ 149,99",149.99,167.24


In [10]:
# interseção das duas tabelas (para retornar apenas as edições de interesse)
cartas_final_df = pd.merge(cartas_df, cartas_web_df, how='left', on=(
    'nome_portugues', 'nome_ingles', 'edicao'))

cartas_final_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 280,33",280.33,312.57
1,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 61,71",61.71,73.81
2,Desenterrar,Unearth,O Legado de Urza,Don Hazeltine,Comum,"R$ 4,76",4.76,10.31
3,Aluren,Aluren,Tempestade,April Lee,Rara,"R$ 338,66",338.66,377.61
4,Tutor Místico,Mystical Tutor,Sexta Edição Classica,David O`Connor,Incomum,"R$ 82,66",82.66,92.17
5,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 49,11",49.11,59.76
6,Aranha Escondida,Hidden Spider,A Saga de Urza,Thomas M. Baxa,Comum,"R$ 3,70",3.70,9.13
7,Guerra Justa,Righteous War,Visões,Ian Miller,Rara,"R$ 26,32",26.32,34.35
8,Bênção de Cho-Manno,Cho-Manno's Blessing,Máscara de Mercádia,John Matson,Comum,"R$ 4,00",4.00,9.46
9,Juramento dos Regentes,Oath of Lieges,Êxodo,Mark Zug,Rara,"R$ 26,23",26.23,34.25


In [11]:
# teste para saber se foi realmente retoranda a lista desejada
if len(cartas_df["nome_portugues"]) == len(cartas_final_df["nome_portugues"]):
    print('Ok!')
else:
    print('Deu errado!')

Ok!


# Carregamento

In [12]:
# gerando o arquivo final
cartas_final_df.to_excel("excel/cartas_magic_output.xlsx")
cartas_final_df

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
0,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 280,33",280.33,312.57
1,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 61,71",61.71,73.81
2,Desenterrar,Unearth,O Legado de Urza,Don Hazeltine,Comum,"R$ 4,76",4.76,10.31
3,Aluren,Aluren,Tempestade,April Lee,Rara,"R$ 338,66",338.66,377.61
4,Tutor Místico,Mystical Tutor,Sexta Edição Classica,David O`Connor,Incomum,"R$ 82,66",82.66,92.17
5,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 49,11",49.11,59.76
6,Aranha Escondida,Hidden Spider,A Saga de Urza,Thomas M. Baxa,Comum,"R$ 3,70",3.70,9.13
7,Guerra Justa,Righteous War,Visões,Ian Miller,Rara,"R$ 26,32",26.32,34.35
8,Bênção de Cho-Manno,Cho-Manno's Blessing,Máscara de Mercádia,John Matson,Comum,"R$ 4,00",4.00,9.46
9,Juramento dos Regentes,Oath of Lieges,Êxodo,Mark Zug,Rara,"R$ 26,23",26.23,34.25


In [13]:
# exibindo as 15 cartas mais caras 
cartas_final_df.sort_values(by='valor_medio', ascending=False).head(15)

,nome_portugues,nome_ingles,edicao,artista,raridade,valor_medio_srt,valor_medio,valor_ml
3,Aluren,Aluren,Tempestade,April Lee,Rara,"R$ 338,66",338.66,377.61
0,Tutor Vampírico,Vampiric Tutor,Sexta Edição Classica,Gary Leach,Rara,"R$ 280,33",280.33,312.57
4,Tutor Místico,Mystical Tutor,Sexta Edição Classica,David O`Connor,Incomum,"R$ 82,66",82.66,92.17
1,Proteção Oscilante,Flickering Ward,Tempestade,Greg Simanson,Incomum,"R$ 61,71",61.71,73.81
5,Abertura Temporal,Temporal Aperture,A Saga de Urza,Michael Sutfin,Rara,"R$ 49,11",49.11,59.76
10,Juramento dos Carnicais,Oath of Ghouls,Êxodo,Brom,Rara,"R$ 40,04",40.04,49.64
7,Guerra Justa,Righteous War,Visões,Ian Miller,Rara,"R$ 26,32",26.32,34.35
9,Juramento dos Regentes,Oath of Lieges,Êxodo,Mark Zug,Rara,"R$ 26,23",26.23,34.25
2,Desenterrar,Unearth,O Legado de Urza,Don Hazeltine,Comum,"R$ 4,76",4.76,10.31
8,Bênção de Cho-Manno,Cho-Manno's Blessing,Máscara de Mercádia,John Matson,Comum,"R$ 4,00",4.00,9.46
